Load Google Drive to Colab

In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/1016/Project/

[Errno 2] No such file or directory: 'drive/MyDrive/1016/Project/'
/content/drive/My Drive/1016/Project/OverFeat


Prerequisite package imagemagick

In [ ]:
!sudo apt-get install imagemagick

Clone the Overfeat Repo to Google Drive

In [ ]:
!git clone https://github.com/sermanet/OverFeat.git

Cloning into 'OverFeat'...
remote: Enumerating objects: 278, done.
remote: Total 278 (delta 0), reused 0 (delta 0), pack-reused 278
Receiving objects: 100% (278/278), 23.76 MiB | 10.62 MiB/s, done.
Resolving deltas: 100% (80/80), done.
Checking out files: 100% (119/119), done.


In [ ]:
%cd OverFeat/

/content/drive/My Drive/1016/Project/OverFeat


Download the weight. This might have problem that Colab reads location 'My Drive' instead of 'MyDrive'. It appears random, sometimes it would read 'MyDrive', sometime with space. Add a replace(' ','') to file path.

In [ ]:
!chmod 755 -R /content/drive/MyDrive/1016/Project/OverFeat

In [ ]:
!python download_weights.py

The instruction said run recompile if OpenBLAS not installed. I did recompile while debugging the directory path problem, might not be necessary.

In [ ]:
%cd src

/content/drive/My Drive/1016/Project/OverFeat/src


In [ ]:
!make all

In [ ]:
!make cam

Set working location to OverFeat folder again

In [ ]:
%cd ..

/content/drive/My Drive/1016/Project/OverFeat


In [ ]:
!pwd

/content/drive/My Drive/1016/Project/OverFeat


Set permission to the OverFeat folder. This was another problem I encountered when trying to run the sample picture.

In [ ]:
!chmod 755 -R /content/drive/MyDrive/1016/Project/OverFeat

Now the Overfeat default example should work

In [ ]:
!bin/linux_64/overfeat -n 6 samples/bee.jpg

bee 0.862415
leaf beetle, chrysomelid 0.0539294
fly 0.0305984
long-horned beetle, longicorn, longicorn beetle 0.0205278
ladybug, ladybeetle, lady beetle, ladybird, ladybird beetle 0.018931
ant, emmet, pismire 0.00558904


In [ ]:
import os
import re

In [ ]:
mypath = os.getcwd() + '/images'
categories = [i for i in os.walk(mypath)][0][1]
categories

['Plane', 'Mug', 'Mountain', 'Lighthouse', 'Church', 'Car', 'Beach', 'Banana']

In [ ]:
for cat in categories:
    dir_path = mypath + '/' + cat
    d = 'images/' + cat
    files = [i[2] for i in os.walk(dir_path)][0]
    files.sort()

    for f in files:
        basename = re.findall(r'[0-9]+',os.path.basename(f))[0]
        input_path = d+'/'+f
        output_path = d.replace('images','output') + '/' + basename + '.txt'
        print(input_path)
        # print(output_path)

        cmd = 'bin/linux_64/overfeat -n 6 {} > {}'.format(input_path,output_path)
        os.system(cmd)

In [ ]:
result_array = []
for cat in categories:
    dir_path = os.getcwd() + '/output/' + cat
    files = [i[2] for i in os.walk(dir_path)][0]
    files.sort()

    for f in files:
        basename = re.findall(r'[0-9]+',os.path.basename(f))[0]
        f_path = dir_path + '/' + f

        file = open(f_path,'r')
        lines = file.readlines()
 
        for line in lines:
            name = re.findall(r'[^0-9]+',line)[0][:-1]
            prob = float(line[len(name)+1:-1])
            result_array.append([cat,basename,name,prob])

In [ ]:
import pandas as pd

In [ ]:
result = pd.DataFrame(result_array)

In [ ]:
result.to_csv('overfeat_results.txt',sep='\t')